In [6]:
cd '/Users/aidasaglinskas/Desktop/2D-and-3D-Deep-Autoencoder/code/'

/Users/aidasaglinskas/Desktop/2D-and-3D-Deep-Autoencoder/code


In [14]:
from lasagne.layers import InputLayer, NINLayer, flatten, reshape, Upscale3DLayer, DenseLayer
from lasagne.layers import get_output, get_output_shape, get_all_params, get_all_layers 
from lasagne.nonlinearities import LeakyRectify as lrelu
from lasagne.nonlinearities import rectify as relu
from lasagne.nonlinearities import sigmoid
from lasagne.objectives import binary_crossentropy as bce
from lasagne.objectives import squared_error
from lasagne.updates import adam

In [15]:
try:
    from lasagne.layers import Conv3DLayer
except ImportError:
    from lasagne.layers.dnn import Conv3DDNNLayer as Conv3DLayer

In [16]:
from TransposedConv3DLayer import TransposedConv3DLayer as Deconv3D

In [18]:
#import theano.tensor.nnet.abstract_conv.BaseAbstractConv

In [19]:
import numpy as np
import theano
from theano import tensor as T
import time
import matplotlib
matplotlib.use('Agg') 
from matplotlib import pyplot as plt
import scipy.io
from scipy import misc
from skimage.io import imsave

print('imported succesfully')

imported succesfully


In [22]:
floatX=theano.config.floatX

inPath = '' #path to dataset

outPath = '' #path to where you want the results to be saved

def get_args():
    print ('getting args...')

def save_args():
    print ('saving args...')

def build_net(nz=200):

    input_depth=160
    input_rows=64
    input_columns=64

    #Encoder 

    enc = InputLayer(shape=(None,1,input_depth,input_rows,input_columns)) #5D tensor
    enc = Conv3DLayer(incoming=enc, num_filters=64, filter_size=5,stride=2, nonlinearity=lrelu(0.2),pad=2)
    enc = Conv3DLayer(incoming=enc, num_filters=128, filter_size=5,stride=2, nonlinearity=lrelu(0.2),pad=2)
    enc = Conv3DLayer(incoming=enc, num_filters=256, filter_size=5,stride=2, nonlinearity=lrelu(0.2),pad=2)
    enc = Conv3DLayer(incoming=enc, num_filters=256, filter_size=5,stride=2, nonlinearity=lrelu(0.2),pad=2)
    enc = reshape(incoming=enc, shape=(-1,256*4*4*10))
    enc = DenseLayer(incoming=enc, num_units=nz, nonlinearity=sigmoid)

    #Decoder 

    dec = InputLayer(shape=(None,nz))
    dec = DenseLayer(incoming=dec, num_units=256*4*4*10)
    dec = reshape(incoming=dec, shape=(-1,256,10,4,4))
    dec=Deconv3D(incoming=dec, num_filters=256, filter_size=4 ,stride=2, crop=1,nonlinearity=relu)
    dec=Deconv3D(incoming=dec, num_filters=128, filter_size=4 ,stride=2, crop=1,nonlinearity=relu)
    dec=Deconv3D(incoming=dec, num_filters=64, filter_size=4 ,stride=2, crop=1,nonlinearity=relu)
    dec=Deconv3D(incoming=dec, num_filters=1, filter_size=4 ,stride=2, crop=1,nonlinearity=sigmoid)



    return enc,dec

In [37]:
infn = '/Users/aidasaglinskas/Desktop/anatomical.npy'
dataArr = np.load(infn)

In [38]:
dataArr.shape

(131, 1, 160, 64, 64)

In [23]:
enc, dec = build_net()
for l in get_all_layers(enc):
    print (get_output_shape(l))
for l in get_all_layers(dec):
    print (get_output_shape(l))

(None, 1, 160, 64, 64)
(None, 64, 80, 32, 32)
(None, 128, 40, 16, 16)
(None, 256, 20, 8, 8)
(None, 256, 10, 4, 4)
(None, 40960)
(None, 200)
(None, 200)
(None, 40960)
(None, 256, 10, 4, 4)
(None, 256, 20, 8, 8)
(None, 128, 40, 16, 16)
(None, 64, 80, 32, 32)
(None, 1, 160, 64, 64)


In [59]:
def prep_train(alpha=0.0002, beta=0.5, nz=200):

    E,D=build_net(nz=nz)

    #x=T.Tensor5('x') 
    x=T.tensor5('x') 
    #x = T.Tensor(dtype='int64',name='x',broadcastable=[True]) # Aidas hax

    # x -> symbolic variable, input to the computational graph

    #Get outputs z=E(x), x_hat=D(z)

    encoding = get_output(E,x)

    decoding = get_output(D,encoding)

    #Get parameters of E and D

    params_e=get_all_params(E, trainable=True)

    params_d=get_all_params(D, trainable=True)

    params = params_e + params_d

    #Calculate cost and updates

    cost = T.mean(squared_error(x,decoding))

    grad=T.grad(cost,params)

    updates = adam(grad,params, learning_rate=alpha , beta1=beta)	

    train = theano.function(inputs=[x], outputs=cost, updates=updates)

    rec = theano.function(inputs=[x], outputs=decoding)

    test = theano.function(inputs=[x], outputs=cost)

    #theano.function returns an actual python function used to evaluate our real data

    return train ,test, rec, E, D

In [26]:
def train( trainData, testData, nz=200, alpha=0.00005, beta=0.5, batchSize=4, epoch=500):

    train, test, rec, E, D = prep_train(nz=nz, alpha=alpha)

    print (np.shape(trainData) )

    sn,sc,sz,sx,sy=np.shape(trainData) 

    print (sn,sc,sz,sx,sy)

    batches=int(np.floor(float(sn)/batchSize))

    #initialize arrays to store the cost functions

    trainCost_=[]

    testCost_=[]

    print ('batches=',batches)

    timer=time.time()

    print ('epoch  \t batch  \t  train  \t  cost  \t    test  \t   cost   \t  time (s)')

    for e in range(epoch):

        for b in range(batches):

            trainCost=train(trainData[b*batchSize:(b+1)*batchSize]) 

            testCost=test(testData[:10]) #test first 10 images

            print (e , '\t', b ,'\t', trainCost ,'\t' ,testCost ,'\t' ,time.time()-timer)

            timer=time.time()

            trainCost_.append(trainCost)

            testCost_.append(testCost)

            #save results every 10 interations

            if b%10==0:

                # create a montage to visualize how close the decoded images are to the input images
                # the chosen method here creates a montage of 2x3 images with 3 real images on top of 3 decoded ones, but other ways to visualize the results can be used

                x_test1=x_test[0:20:7,:,:,:]

                montageRow1 = np.hstack(x_test1[:3].reshape(-1,160,64,64).swapaxes(1,3))

                REC=rec(x_test1[:3,:,:,:,:])

                montageRow2 = np.hstack(REC[:3].reshape(-1,160,64,64).swapaxes(1,3))

                montage = np.vstack((montageRow1, montageRow2))

                scipy.io.savemat(outPath + 'montageREC'+str(e)+'.mat',(dict(montage=montage)))


            # save plot of the cost functions

            plt.plot(range(e*batches+b),trainCost_[:e*batches + b])

            plt.plot(range(e*batches+b),testCost_[:e*batches + b]) 

            plt.legend()

            plt.xlabel('Iterations')

            plt.ylabel('Cost Function')

            plt.savefig(outPath + 'cost_regular_{}.png'.format(e))

    return test, rec, E, D

In [46]:
def test(x, rec):

    return rec(x)

# LOAD DATA

data=np.load(inPath,mmap_mode='r').astype(floatX)

data=np.transpose(data,(0,2,1,3,4))

sn,sc,sz,sx,sy=np.shape(data) 

# normalize input dataset

for n in range(sn):

    for j in range(sz):

        data[n,:,j,:,:]=data[n,:,j,:,:]/data[n,:,j,:,:].max()

# create training and testing datasets

x_train=data[20:,:,:,:,:]

x_test=data[:20,:,:,:,:]

test, rec, E, D =train(x_train, x_test) 

# Save example reconstructions at the end of the iterations

REC = rec(x_test[:10])

print (np.shape(REC), np.shape(x_test[:10]))

fig=plt.figure()

newDir=''

montageRow1 = np.hstack(x_test[:3].reshape(-1,160,64,64).swapaxes(1,3))

montageRow2 = np.hstack(REC[:3].reshape(-1,160,64,64).swapaxes(1,3))

montage = np.vstack((montageRow1, montageRow2))

print ('montage shape is ',montage.shape)

np.save(outPath + 'montageREC.npy',montage)

scipy.io.savemat(outPath + 'montageREC.mat',(dict(montage=montage))) 

FileNotFoundError: [Errno 2] No such file or directory: ''

In [60]:
trainData = dataArr[1:100,:,:,:]
testData = dataArr[100:130,:,:,:]
train( trainData, testData, nz=200, alpha=0.00005, beta=0.5, batchSize=4, epoch=500)

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1, 1, .. 1) of 0.5})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/cc.py",


You can find the C code in this temporary file: /var/folders/m7/h5wbg21x40sfshpzz979230c0000gn/T/theano_compilation_error_m25ilzx9


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1) of 0.5})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/cc.py", line 121


You can find the C code in this temporary file: /var/folders/m7/h5wbg21x40sfshpzz979230c0000gn/T/theano_compilation_error_9cal9tat


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1, 1, .. 1) of 0.5})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/cc.py",

...

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1, 1, .. 1) of 0.5})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/cc.py",


You can find the C code in this temporary file: /var/folders/m7/h5wbg21x40sfshpzz979230c0000gn/T/theano_compilation_error_pj_7h13y


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1) of 0.5})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/op.py", line 858, in make_c_thunk
    output_storage=node_output_storage)
  File "/Users/aidasaglinskas/opt/anaconda3/envs/py3/lib/python3.7/site-packages/theano/gof/cc.py", line 121


You can find the C code in this temporary file: /var/folders/m7/h5wbg21x40sfshpzz979230c0000gn/T/theano_compilation_error_rd318yx4

You can find the C code in this temporary file: /var/folders/m7/h5wbg21x40sfshpzz979230c0000gn/T/theano_compilation_error_kut3v0y_


Exception: ('The following error happened while compiling the node', Elemwise{Cast{float64}}(TensorConstant{(1, 1, 1, .. 1) of 0.5}), '\n', "Compilation failed (return status=1): /Users/aidasaglinskas/.theano/compiledir_Darwin-19.4.0-x86_64-i386-64bit-i386-3.7.7-64/tmp8i9f_zwb/mod.cpp:414:27: error: non-constant-expression cannot be narrowed from type 'npy_intp' (aka 'long') to 'int' in initializer list [-Wc++11-narrowing].     int init_totals[5] = {V1_n0, V1_n1, V1_n2, V1_n3, V1_n4};.                           ^~~~~. /Users/aidasaglinskas/.theano/compiledir_Darwin-19.4.0-x86_64-i386-64bit-i386-3.7.7-64/tmp8i9f_zwb/mod.cpp:414:27: note: insert an explicit cast to silence this issue.     int init_totals[5] = {V1_n0, V1_n1, V1_n2, V1_n3, V1_n4};.                           ^~~~~.                           static_cast<int>( ). /Users/aidasaglinskas/.theano/compiledir_Darwin-19.4.0-x86_64-i386-64bit-i386-3.7.7-64/tmp8i9f_zwb/mod.cpp:414:34: error: non-constant-expression cannot be narrowed 